In [115]:
from playwright.async_api import async_playwright



## Define Variables


In [116]:
LOGIN_PAGE_URL = "https://www.beenverified.com/app/login"
HOME_PAGE_URL = "https://www.beenverified.com/rf/dashboard"
REPORT_PAGE_URL = "https://www.beenverified.com/api/v5/reports"
SEARCH_PAGE_URL = "https://www.beenverified.com/rf/search/person"

In [117]:
pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)



In [118]:
context = await browser.new_context(storage_state="playwright/.auth/state.json")
page =  await context.new_page()
await page.goto(HOME_PAGE_URL)


<Response url='https://www.beenverified.com/rf/dashboard' request=<Request url='https://www.beenverified.com/rf/dashboard' method='GET'>>

## Search for lead 

In [119]:
def build_search_url(first_name, last_name, city: str | None = None, state :str | None = None, age: int | None = None):
    url = f"{SEARCH_PAGE_URL}?age={age}&city={city}&fname={first_name}&ln={last_name}&mn=&state={state}"
    return url


In [120]:
search_query = {
    "first_name": "Samama",
    "last_name": "Mahmud",
    "city": "",
    "state": "",
    "age": None
}

url = build_search_url(search_query["first_name"], search_query["last_name"], search_query["city"], search_query["state"], search_query["age"])
await page.goto(url)

<Response url='https://www.beenverified.com/rf/search/person?age=None&city=&fname=Samama&ln=Mahmud&mn=&state=' request=<Request url='https://www.beenverified.com/rf/search/person?age=None&city=&fname=Samama&ln=Mahmud&mn=&state=' method='GET'>>

# get top 5 users

In [137]:
def score_of_leads(result, search_query):
    score = 0

    if search_query.get("first_name").lower() not in result.lower() or search_query.get("last_name").lower() not in result.lower() or "deceased" in result.lower():
        score = -1
        return score

    score += result.lower().count("alias")
    score += result.lower().count("relatives")
    score += result.lower().count(search_query.get("city").lower() if search_query.get("city") else "")
    score += result.lower().count(search_query.get("state").lower() if search_query.get("state") else "")
    score += result.lower().count(search_query.get("age").lower() if search_query.get("age") else "")
    score += result.lower().count(search_query.get("middle_name").lower() if search_query.get("middle_name") else "")

    return score


In [122]:
container_results = await page.query_selector(".css-ts1zsd")
container_results

<JSHandle preview=JSHandle@node>

In [123]:
container_users = await container_results.query_selector_all(".css-1mvdt3q")
#len of leads matched
len(container_users)

1

## filter by score lead


In [133]:
txt = await container_users[0].text_content()
txt

'Name matchSamama B Mahmud, 28Morton Grove, ILBornJul 1995RelativesAbdullah Al Mahmud, Labiba B Mahmud, Sharif Mahmud, +2 MoreLocationsMorton Grove, IL • Milwaukee, WIView Person Report'

In [138]:
#score of leads
score_of_leads(txt, search_query)


741

In [139]:
for lead in container_users:
    print(await lead.text_content())
    print("----")

Name matchSamama B Mahmud, 28Morton Grove, ILBornJul 1995RelativesAbdullah Al Mahmud, Labiba B Mahmud, Sharif Mahmud, +2 MoreLocationsMorton Grove, IL • Milwaukee, WIView Person Report
----


## Get info leasd

In [70]:
async def open_report_popup(page, container_user):
    async with page.expect_popup() as popup_info:
        view_report = await container_user.query_selector(".css-gnnc3j")
        await view_report.click()
        new_page = await popup_info.value
        return new_page


In [71]:
async def open_report_page(page, container_user):
    async with page.expect_popup() as popup_info:
        view_report = await container_user.query_selector(".css-gnnc3j")
        await view_report.click()
        new_page = await popup_info.value
        return new_page



# Exetra information of the user

In [74]:
async def extract_phone_numbers(page):
    container_phone = await page.query_selector("#phone-numbers-section")
    phone_numbers = await container_phone.query_selector_all(".css-1vugsqn")
    phone_numbers = [await phone_number.text_content() for phone_number in phone_numbers]
    return phone_numbers




In [75]:
async def extract_addresses(page):
    addresses_list = []
    container_address = await page.query_selector("#address-history-section")
    addresses = await container_address.query_selector_all(".css-1q4wjho")
    for address in addresses:
        address_fields = await address.query_selector_all(".css-zv7ju9")
        addresses_txt = [await address_field.text_content() for address_field in address_fields]
        addresses_list.append(' '.join(addresses_txt))
        
    return addresses_list



In [105]:
async def extract_email_list(page):
   
    email_container = await page.query_selector("#email-section")
    email_elements = await email_container.query_selector_all(".css-1vugsqn")
    email_list = [await email_element.text_content() for email_element in email_elements]
    return email_list



In [109]:
async def get_lead_info(new_page):
    extra_phone_numbers = await extract_phone_numbers(new_page)  # noqa: F821
    extra_addresses = await extract_addresses(new_page)
    extra_emails = await extract_email_list(new_page)
    return extra_phone_numbers, extra_addresses, extra_emails

# extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
# extra_phone_numbers, extra_addresses, extra_emails

    
    
    

## Get info of the user

In [110]:
for lead in container_users:
    print(await lead.text_content())
    new_page = await open_report_page(page, lead)
    await new_page.wait_for_selector("#person-overview")
    extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
    print(extra_phone_numbers, extra_addresses, extra_emails)
    print("----")
    # await new_page.close()

Name matchSamama B Mahmud, 28Morton Grove, ILBornJul 1995RelativesAbdullah Al Mahmud, Labiba B Mahmud, Sharif Mahmud, +2 MoreLocationsMorton Grove, IL • Milwaukee, WIView Person Report
JSHandle@<button tabindex="0" type="button" class="MuiButtonBase-…>…</button>
['(847) 521-9814', '(847) 581-0171'] ['5808 Keeney St Morton Grove, IL 60053', '811 22nd St Apt 11 Milwaukee, WI 53233'] ['samamamahmud17@gmail.com', 'samamamahmud@yahoo.com', 'samama.mahmud@marquette.edu']
----


In [102]:
extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
print(extra_phone_numbers, extra_addresses, extra_emails)

['(847) 521-9814', '(847) 581-0171'] ['5808 Keeney St Morton Grove, IL 60053', '811 22nd St Apt 11 Milwaukee, WI 53233'] ['samamamahmud17@gmail.com', 'samamamahmud@yahoo.com', 'samama.mahmud@marquette.edu']


In [112]:
await new_page.close()

In [113]:
await browser.close()
await pw.stop()